# F1 Elo Prototyping

Notebook for exploring ideas before binning or porting to scripts.

## Environment setup

In [6]:
import yaml

import pandas as pd
import plotly.express as px

In [2]:
# Move project root folder
%cd ..

/Users/mwtmurphy/projects/personal/f1-elo


/Users/mwtmurphy/Library/Caches/pypoetry/virtualenvs/f1-elo-GZG1X434-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Data importing

In [21]:
with open("params.yaml") as conf_file:
    CONFIG = yaml.safe_load(conf_file)

mod_df = pd.read_csv(CONFIG["data"]["modelled_path"])
con_df = pd.read_csv(CONFIG["data"]["constructors_csv"]).rename(columns={"name": "constructorName"})
dri_df = pd.read_csv(CONFIG["data"]["drivers_csv"])
dri_df["driverName"] = dri_df[["forename", "surname"]].apply(lambda row: " ".join(row), axis=1)
vis_df = mod_df.merge(dri_df[["driverId", "driverName"]], on=["driverId"], how="left")
vis_df = vis_df.merge(con_df[["constructorId", "constructorName"]], on="constructorId", how="left") 
vis_df.head()

,year,round,date,constructorId,driverId,mapPosition,mapPoints,status,constructorScore,driverScore,driverName,constructorName
0,1950,1,1950-05-13,51,642,1,25.0,finished,1548.446999,1559.212999,Nino Farina,Alfa Romeo
1,1950,1,1950-05-13,51,786,2,18.0,finished,1548.446999,1548.446999,Luigi Fagioli,Alfa Romeo
2,1950,1,1950-05-13,51,686,3,15.0,finished,1548.446999,1537.680999,Reg Parnell,Alfa Romeo
3,1950,1,1950-05-13,154,704,4,12.0,finished,1491.925500,1526.914999,Yves Cabantous,Talbot-Lago
4,1950,1,1950-05-13,154,627,5,10.0,finished,1491.925500,1516.149000,Louis Rosier,Talbot-Lago


## Data visualisation

In [16]:
cur_dri = set(vis_df.loc[vis_df["year"] == 2023, "driverId"])
cur_df = vis_df[vis_df["driverId"].isin(cur_dri)]

px.line(cur_df, x="date", y="driverScore", color="driverName")

In [17]:
yr, rnd = cur_df.iloc[-1, :2]
cur_df.loc[(cur_df["year"] == yr) & (cur_df["round"] == rnd), ["driverName", "driverScore"]].sort_values("driverScore", ascending=False)

,driverName,driverScore
26220,Max Verstappen,1905.899722
26221,Lando Norris,1769.990513
26222,Fernando Alonso,1727.250650
26225,Carlos Sainz,1691.488807
26227,Lewis Hamilton,1683.428283
26234,Charles Leclerc,1671.841042
26226,Pierre Gasly,1620.337443
26237,Alexander Albon,1610.722098
26223,Sergio Pérez,1576.138608
26231,Nico Hülkenberg,1564.412222


In [18]:
cur_con = set(vis_df.loc[vis_df["year"] == 2023, "constructorId"])
cur_df = vis_df[vis_df["constructorId"].isin(cur_con)]

px.line(cur_df, x="date", y="constructorScore", color="constructorName")

In [20]:
yr, rnd = cur_df.iloc[-1, :2]
cur_df.loc[(cur_df["year"] == yr) & (cur_df["round"] == rnd), ["constructorName", "constructorScore"]].sort_values("constructorScore", ascending=False).drop_duplicates()

,constructorName,constructorScore
26220,Red Bull,1826.533695
26235,Mercedes,1692.346666
26221,McLaren,1653.532238
26225,Ferrari,1651.963573
26222,Aston Martin,1576.861597
26226,Alpine F1 Team,1557.244737
26228,AlphaTauri,1526.356614
26238,Alfa Romeo,1459.092000
26237,Williams,1455.530435
26236,Haas F1 Team,1439.427205
